In [1]:
# import os
# import sys
# import time as time

# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# current_file_path = os.getcwd()
# src_path = os.path.abspath(os.path.join(current_file_path, "..", "src"))
# print(f"src path: {src_path}")
# sys.path.insert(0, src_path)

# import torch
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
import os, sys

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# 获取当前文件的绝对路径
current_file_path = os.getcwd()

# 跳出 g_unet 目录，进入 Xinyi_GNN_aorta/src
project_root = os.path.abspath(os.path.join(current_file_path, ".."))
src_path = os.path.join(project_root, "src")

print(f"src path: {src_path}")
sys.path.insert(0, src_path)

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

src path: /ehome/xinyi/Xinyi_GNN_aorta/src


device(type='cuda')

In [3]:
# from utils import aorta_3D_info, Normalizer_ts,vtp2Graph_aorta

# # In this code block we will derive the data information for the aorta_toy dataset.
# raw_data_path = '/ehome/xinyi/Xinyi_GNN_aorta/data/toy_trans/raw'
# outputpath = '/ehome/xinyi/Xinyi_GNN_aorta/data/toy_trans'
# data_label = ['y', 'pos', 'stmdist']
# data_information = aorta_3D_info(path=raw_data_path, labels=data_label, output_path=outputpath)


In [4]:
from utils import aorta_3D_info, Normalizer_ts

processed_path = '/ehome/xinyi/Xinyi_GNN_aorta/data/toy_trans/raw'
outputpath     = '/ehome/xinyi/Xinyi_GNN_aorta/data/toy_trans'
data_label = ['y', 'pos', "stmdist"]

data_information = aorta_3D_info(path=processed_path, labels=data_label, output_path=outputpath)
dfs  = data_information.graph2pandas()


Generating data info...


100%|██████████| 3/3 [00:00<00:00, 686.88it/s]

data generation done, saved in directory:/ehome/xinyi/Xinyi_GNN_aorta/data/toy_trans


In [5]:
data_information.info

,max,min,mean,std
label,,,,
pressure,3.398470,-7.650037,1.278087,1.304616
wssx,0.021710,-0.046168,-0.002896,0.006476
wssy,0.010297,-0.074118,-0.008381,0.009542
wssz,0.079113,-0.036755,0.006116,0.017247
x,38.824947,-17.211851,15.181118,13.173093
y,46.847961,-48.988758,-3.707316,24.354790
z,82.896675,-5.897548,42.835144,18.339022
distance,177.199799,0.000000,67.495743,45.188190


In [6]:
# import tqdm
# import torch
# import numpy as np

# # ✅ 正确使用从 data_information 提取的 mean/std
# output_range = data_information.info.values[1].astype(np.float32)
# pressure_mean = output_range[2]  # e.g. 1.5284
# pressure_std = output_range[3]   # e.g. 2.1039

# print("✅ Using global mean/std from data_information:")
# print(f"Mean = {pressure_mean:.4f}, Std = {pressure_std:.4f}")

# output_normalizer = Normalizer_ts(method="ms", params=np.array([[pressure_mean], [pressure_std]]))

# raw_paths = ['/ehome/xinyi/Xinyi_GNN_aorta/data/toy_trans/raw/aorta0000.vtp']

# for raw_path in tqdm.tqdm(raw_paths):
#     data, _ = vtp2Graph_aorta(raw_path)

#     print(f"📂 File: {raw_path}")
#     print("Before normalization:")
#     print(f"  min = {data.y[:, 0].min().item():.4f}")
#     print(f"  max = {data.y[:, 0].max().item():.4f}")
#     print(f"  mean = {data.y[:, 0].mean().item():.4f}")
#     print(f"  std  = {data.y[:, 0].std().item():.4f}")

#     data.y = output_normalizer.normalize(data.y[:, 0])

#     print("After normalization:")
#     print(f"  min = {data.y.min().item():.4f}")
#     print(f"  max = {data.y.max().item():.4f}")
#     print(f"  mean = {data.y.mean().item():.4f}")
#     print(f"  std  = {data.y.std().item():.4f}")


In [7]:
# import numpy as np

# # Extracting output and input ranges from the data information for data normalization
# output_range = data_information.info.values[1].reshape(1, -1) # pressure!!!!!!!!!!!!!
# input_range = data_information.info.values[4:] 


# output_range = output_range.astype(np.float32)
# input_range = input_range.astype(np.float32)

In [8]:
import numpy as np

# Extracting output ranges from the data information for data normalization

output_range = data_information.info.values[0].reshape(1, -1) # pressure!!!!!!!!!!!!!
output_range = output_range.astype(np.float32)

# Step 1: 提取 x, y, z 三行的数据（索引为 4, 5, 6）
xyz_rows = data_information.info.values[4:7].astype(np.float32)

# Step 2: 提取 mean 和 std 列（索引 2 和 3）
xyz_mean = xyz_rows[:, 2]  # shape = (3,)
xyz_std = xyz_rows[:, 3]   # shape = (3,)

# Step 3: 计算 mean 和 std 的平均值
shared_mean = np.mean(xyz_mean)
shared_std = np.mean(xyz_std)

# Step 4: 构造新的 mean/std 参数，用于 x/y/z 和其他维度（比如 distance）拼接
# x/y/z 使用统一的 mean/std，distance 用原来的
distance_row = data_information.info.values[7].astype(np.float32)  # index 7 是 distance

# 构造 input normalization 参数，按列顺序排列
# shape = (2, 4) → [mean_row, std_row]
input_mean = np.array([shared_mean, shared_mean, shared_mean, distance_row[2]])
input_std = np.array([shared_std, shared_std, shared_std, distance_row[3]])
input_params = np.vstack([input_mean, input_std])  # shape = (2, 4)

In [9]:
# input_normalizer = Normalizer_ts(method='ms',params=input_range[:,2:].T)
input_normalizer = Normalizer_ts(method='ms', params=input_params)
output_normalizer = Normalizer_ts(method="ms", params=output_range[:,2:].T)

In [10]:
# from torch_cluster import fps, knn
# @torch.no_grad()
# def positional_encoding(data):

#     vectors_to = {key: data.pos[value.long()] - data.pos for key, value in compute_nearest_boundary_vertex(data).items()}
#     distances_to = {key: torch.linalg.norm(value, dim=-1, keepdim=True) for key, value in vectors_to.items()}

#     data.x = torch.cat((
#         vectors_to['inlet'] / torch.clamp(distances_to['inlet'], min=1e-16),
#         vectors_to['lumen_wall'] / torch.clamp(distances_to['lumen_wall'], min=1e-16),
#         vectors_to['outlets'] / torch.clamp(distances_to['outlets'], min=1e-16),
#         distances_to['inlet'],
#         distances_to['lumen_wall'],
#         distances_to['outlets']
#     ), dim=1)

#     return data

# def compute_nearest_boundary_vertex(data):
#     index_dict = {}

#     for key in ('inlet', 'lumen_wall', 'outlets'):
#         index_dict[key] = data[f'{key}_index'][knn(data.pos[data[f'{key}_index'].long()], data.pos, k=1)[1].long()]

#     return index_dict

In [11]:
from dataset_copy import Aorta_3d_Dataset
from lab_gatr.transforms import PointCloudPoolingScales

import torch_geometric.transforms as T

# 1. 定义采样比例和插值简单形
sampling_ratios = (0.1,)
interpolation_simplex = 'triangle'

# 2. 正确地创建 PointCloudPoolingScales 实例
#    传入 rel_sampling_ratios 和 interp_simplex 参数
transforms = PointCloudPoolingScales(rel_sampling_ratios=sampling_ratios, interp_simplex=interpolation_simplex)

root = '/ehome/xinyi/Xinyi_GNN_aorta/data/toy_trans'
dataset = Aorta_3d_Dataset(root,'cpu', label='pressure', pre_transform=transforms,input_normalizer=input_normalizer,output_normalizer=output_normalizer)

Processing...
100%|██████████| 11/11 [00:00<00:00, 18.58it/s]
Done!


In [12]:
# import os
# import numpy as np
# import torch
# import pyvista as pv

# # 取一个样本
# data = dataset[0]

# # -------- 1) 选择用于可视化的坐标（真实几何）---------
# # 若你在process中保存了pos_raw，就优先使用它；否则退回pos
# points = data.pos_raw.cpu().numpy() if hasattr(data, 'pos_raw') else data.pos.cpu().numpy()
# N = points.shape[0]

# # -------- 2) 构造三角面片（若有）---------
# faces = None
# if hasattr(data, 'face') and data.face is not None:
#     # data.face 通常是形状 [3, F] 的LongTensor（每列一个三角形）
#     face_np = data.face.t().cpu().numpy()  # (F, 3)
#     F = face_np.shape[0]
#     # VTK faces需要 [3, i0, i1, i2, 3, j0, j1, j2, ...]
#     faces = np.hstack([np.full((F, 1), 3, dtype=np.int64), face_np]).ravel()

# # -------- 3) 构建原始网格 PolyData ---------
# if faces is not None:
#     mesh = pv.PolyData(points, faces)
# else:
#     # 没有面片也可以只保存点云（ParaView里以 Points 模式显示）
#     mesh = pv.PolyData(points)

# # -------- 4) 写入各尺度的采样标记到点属性 ---------
# # 自动发现所有 scale{i}_sampling_index
# scale_keys = sorted([k for k in data.keys() if k.endswith('sampling_index')])
# print("Found sampling keys:", scale_keys)

# for sk in scale_keys:
#     # sk 形如 'scale0_sampling_index', 'scale1_sampling_index', ...
#     idxs = data[sk].cpu().numpy().astype(np.int64)   # (M,)
#     mask = np.zeros((N,), dtype=np.int8)
#     mask[idxs] = 1
#     # 点属性名：例如 'scale0_mask'
#     attr_name = sk.replace('_sampling_index', '_mask')
#     mesh.point_data[attr_name] = mask

#     # 同时导出该尺度的点云（只包含被采样的点），便于单独叠加查看
#     sampled_pts = points[idxs]
#     sampled_cloud = pv.PolyData(sampled_pts)
#     # 可选：给点云一个常量标量，方便 ParaView 着色
#     sampled_cloud.point_data['ones'] = np.ones((sampled_pts.shape[0],), dtype=np.float32)

#     # 保存独立点云
#     out_points_vtp = f'aorta_{sk}_points.vtp'
#     sampled_cloud.save(out_points_vtp)
#     print(f"Saved sampled points: {out_points_vtp}")

# # -------- 5) 保存带多尺度mask的完整网格 ---------
# out_mesh_vtp = 'aorta_with_scales_mask.vtp'
# mesh.save(out_mesh_vtp)
# print(f"Saved mesh with masks: {out_mesh_vtp}")


In [13]:
# print(dataset[0])
data = dataset[0].to(device)
mean = data.y.mean()
std = data.y.std()

print(f"Mean: {mean.item():.4f}, Std: {std.item():.4f}")

Mean: 0.1721, Std: 0.7942


In [14]:
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split

total = len(dataset)
train_size = int(0.9 * total)
valid_size = int(0.1 * total)
test_size  = total - train_size

train_dataset, test_dataset= random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(0))
_, valid_dataset = random_split(train_dataset, [train_size-valid_size, valid_size], generator=torch.Generator().manual_seed(1))

train_loader = DataLoader(train_dataset, batch_size = 4, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size = 1)
test_loader = DataLoader(test_dataset, batch_size = 1)

In [15]:
# from lab_gatr.nn.gnn import PointCloudPooling, pool
# my_mlp = torch.nn.Linear(6, 128)
# my_pool = PointCloudPooling(my_mlp)
# pool(my_pool, dataset[0].pos, dataset[0].pos, dataset[0], 0)

In [16]:
# from torch.utils.data import random_split
# from torch_geometric.loader import DataLoader

# # —— 打印数据集大小 —— 
# print("Total dataset size:", len(dataset))

# # —— 固定随机种子 —— 
# manual_seed = 42
# generator = torch.Generator().manual_seed(manual_seed)

# # —— 第一步：10:1 分出 test —— 
# total_size = len(dataset)
# test_size = int(1/11 * total_size)
# train_val_size = total_size - test_size

# train_val_dataset, test_dataset = random_split(
#     dataset,
#     [train_val_size, test_size],
#     generator=generator
# )

# # —— 第二步：从 train_val 中再分 1/10 给 valid —— 
# valid_size = int(train_val_size * 0.1)
# train_size = train_val_size - valid_size

# train_dataset, valid_dataset = random_split(
#     train_val_dataset,
#     [train_size, valid_size],
#     generator=generator
# )

# # —— 构造 DataLoader —— 
# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
# valid_loader = DataLoader(valid_dataset, batch_size=1)
# test_loader  = DataLoader(test_dataset, batch_size=1)

# # —— 打印最终大小 —— 
# print(f"Train: {len(train_dataset)}, Valid: {len(valid_dataset)}, Test: {len(test_dataset)}")

In [17]:
# from lab_gatr import LaBGATr  # ← 根据你的实际文件名改
# from gatr.interface import embed_oriented_plane
# from torch.optim.lr_scheduler import ExponentialLR

# # 假设你已加载好 dataset（例如 DataLoader 里拿一个 sample）
# sample = dataset[0]

# # 准备几何代数接口
# geometric_algebra_interface = embed_oriented_plane(
#     sample.x,
#     num_input_scalars=sample.x.size(-1),  # 自动推断输入标量数
#     num_output_scalars=sample.y.size(-1)  # 自动推断输出标量数
# )

# # 通道参数自动确定
# in_ch = sample.x.size(-1)
# out_ch = sample.y.size(-1)

# # 初始化 LaBGATr 模型
# model = LaBGATr(
#     geometric_algebra_interface=geometric_algebra_interface,
#     d_model=64,
#     num_blocks=4,
#     num_attn_heads=4,
#     num_latent_channels=64,
#     use_class_token=False,
#     dropout_probability=0.1,
#     pooling_mode='cross_attention'  # 或 'message_passing'
# )

# model = model.to('cuda')  # 使用 GPU

# # 损失函数与优化器
# lr = 0.0001
# criterion = torch.nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
# schedule = ExponentialLR(optimizer, gamma=0.999)

# # 训练状态保存字典
# optim_para = {
#     'epoch': 0,
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(),
#     'loss': 1,
# }


In [18]:
# from gatr.interface import embed_point, embed_oriented_plane, extract_oriented_plane
# class GeometricAlgebraInterface:
#     num_input_channels = 1 + 3  # vertex positions plus positional encoding vectors
#     num_output_channels = 1

#     num_input_scalars = 16  # positional encoding sclars
#     num_output_scalars = 4

#     @staticmethod
#     @torch.no_grad()
#     def embed(data):

#         multivectors = torch.cat((
#             embed_point(data.pos).view(-1, 1, 16),
#             *(embed_oriented_plane(data.x[:, slice(i * 3, i * 3 + 3)], data.pos).view(-1, 1, 16) for i in range(3))
#         ), dim=1)
#         scalars = data.x[:, 9:]

#         return multivectors, scalars

#     @staticmethod
#     def dislodge(multivectors, scalars):
#         return extract_oriented_plane(multivectors).squeeze()

In [19]:
from gatr.interface import embed_oriented_plane, extract_translation

class GeometricAlgebraInterface:
    num_input_channels = num_output_channels = 1
    num_input_scalars = num_output_scalars = 1

    @staticmethod
    @torch.no_grad()
    def embed(data):

        multivectors = embed_oriented_plane(normal=data.orientation, position=data.pos).view(-1, 1, 16)
        scalars = data.scalar_feature.view(-1, 1)

        return multivectors, scalars

    @staticmethod
    def dislodge(multivectors, scalars):
        return extract_translation(multivectors).squeeze()

In [20]:
# from gatr.interface import embed_point, embed_oriented_plane, extract_oriented_plane

# class GeometricAlgebraInterface:
#     num_input_channels = 1 + 3  # vertex positions plus positional encoding vectors
#     num_output_channels = 1

#     num_input_scalars = 3  # positional encoding sclars
#     num_output_scalars = 1

#     @staticmethod
#     @torch.no_grad()
#     def embed(data):

#         multivectors = torch.cat((
#             embed_point(data.pos).view(-1, 1, 16),
#             *(embed_oriented_plane(data.x[:, slice(i * 3, i * 3 + 3)], data.pos).view(-1, 1, 16) for i in range(3))
#         ), dim=1)
#         scalars = data.x[:, 9:]

#         return multivectors, scalars

#     @staticmethod
#     def dislodge(multivectors, scalars):
#         return extract_oriented_plane(multivectors).squeeze()

In [21]:
# import sys
# sys.path.insert(0, '../lab_gatr')
# from nn.gnn import PointCloudPooling, pool  

# my_mlp = torch.nn.Linear(6, 128)
# my_pool = PointCloudPooling(my_mlp)
# pool(my_pool, my_dataset[0].pos, my_dataset[0].pos, my_dataset[0], 0)

In [22]:
from lab_gatr.models.lab_gatr import LaBGATr
from torch.optim.lr_scheduler import ExponentialLR

geo_intf = GeometricAlgebraInterface  # 你定义的那个 class

model = LaBGATr(
    geometric_algebra_interface=geo_intf,
    d_model=64,
    num_blocks=4,
    num_attn_heads=4,
    num_latent_channels=48,   # 必须偶数
    use_class_token=False,
    dropout_probability=0.1,
).to(device)



lr = 0.001

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr,weight_decay=0.001)
schedule = ExponentialLR(optimizer,.999)

optim_para = {
            'epoch': 0,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss':1,
            }

LaB-GATr (2121104 parameters)


In [23]:
# with torch.no_grad():
#     # 取一小批样本
#     sample = next(iter(train_loader))
#     # print(sample)
#     # sample = dataset[0].to(device)
#     out = model(sample.to(device))
#     print(out.shape)

In [24]:
# import sys
# sys.path.insert(0, '/ehome/xinyi/Suk/lab-gatr-test/src')

# from tqdm import tqdm
# from train import train, valid

# import matplotlib.pyplot as plt
# #start training 
# train_epoch_error = []
# val_epoch_error = []
# test_error = []
# check_idx = []
# epochs = 1000

# for epoch in tqdm(range(epochs)):
#     train_epoch_error.append(train(model, device, train_loader, optimizer,criterion,scheduler=schedule))
#     val_epoch_error.append(valid(model, device, valid_loader,criterion))
#     if (epoch+1)%100==0 or epoch==0:
#         # val_epoch_error.append(valid(model, device, valid_loader,criterion))
#         test_error.append(valid(model, device, test_loader,criterion))
#         check_idx.append(epoch)
#     # if optim_para['loss'] > train_epoch_error[epoch]:
#     #     optim_para['epoch'] = epoch
#     #     optim_para['model_state_dict'] = model.state_dict()
#     #     optim_para['optimizer_state_dict']= optimizer.state_dict()
#     #     optim_para['loss'] = train_epoch_error[epoch]
#     # check_idx.append(epoch+1)


#     if (epoch+1)%100==0 and epoch != 0:    
#         fig, ax = plt.subplots(figsize=(10,8))
#         er1 = torch.stack(train_epoch_error)
#         er2 = torch.stack(val_epoch_error)
#         er3 = torch.stack(test_error)
#         ax.plot(er1.detach().cpu(),color = 'C0',linestyle='solid',linewidth=1,alpha=1,label='L_train')
#         ax.plot(er2.detach().cpu(),color = 'C1',linestyle='solid',linewidth=1,alpha=1,label='L_valid')
#         ax.plot(check_idx,er3.detach().cpu(),color = 'C2',linestyle='solid',linewidth=1,alpha=1,label='L_test')
#         ax.set_yscale('log')
#         ax.set_title('train_loss_curve')
#         ax.set_xlabel('epochs')
#         ax.set_ylabel('loss')
#         ax.legend()
#         plt.show()

In [25]:
from tqdm import tqdm
from train import train, valid

import torch
import matplotlib.pyplot as plt
import os

# ✅ checkpoint 路径
ckpt_dir = 'checkpoints_toy_vs_sgnn'
os.makedirs(ckpt_dir, exist_ok=True)

# ✅ 图像保存路径
fig_dir = 'figures_toy_vs_sgnn'
os.makedirs(fig_dir, exist_ok=True)

# ✅ 初始化日志
train_epoch_error = []
val_epoch_error = []
test_epoch_error = []
check_idx = []

epochs = 1000

# ✅ 开始训练
for epoch in tqdm(range(epochs)):
    train_loss = train(model, device, train_loader, optimizer, criterion, scheduler=schedule)
    train_epoch_error.append(train_loss)

    if (epoch  % 100) == 0:
        val_loss = valid(model, device, valid_loader, criterion)
        test_loss = valid(model, device, test_loader, criterion)

        val_epoch_error.append(val_loss)
        test_epoch_error.append(test_loss)
        check_idx.append(epoch)

        # ✅ 保存 checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'test_loss': test_loss
        }, os.path.join(ckpt_dir, f'checkpoint_epoch_{epoch}.pt'))

        # ✅ 画图 1：训练损失
        fig1, ax1 = plt.subplots(figsize=(8, 5))
        ax1.plot(
            range(len(train_epoch_error)),
            torch.stack(train_epoch_error).detach().cpu().numpy(),
            color='C0', label='Train Loss'
        )
        ax1.set_yscale('log')
        ax1.set_title('Train Loss Curve')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        fig1.tight_layout()
        fig1.savefig(os.path.join(fig_dir, f"train_loss_epoch_{epoch}.png"))
        plt.close(fig1)

        # ✅ 画图 2：Val/Test 损失（每 100 epoch）
        fig2, ax2 = plt.subplots(figsize=(8, 5))
        ax2.plot(
            check_idx,
            torch.stack(val_epoch_error).detach().cpu().numpy(),
            color='C1', label='Validation Loss'
        )
        ax2.plot(
            check_idx,
            torch.stack(test_epoch_error).detach().cpu().numpy(),
            color='C2', label='Test Loss'
        )
        ax2.set_yscale('log')
        ax2.set_title('Validation & Test Loss (every 100 epochs)')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Loss')
        ax2.legend()
        fig2.tight_layout()
        fig2.savefig(os.path.join(fig_dir, f"val_test_loss_epoch_{epoch}.png"))
        plt.close(fig2)

  0%|          | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_2416088/803710104.py:18: GATrDeprecationWarning: The function "extract_translation" is deprecated, because it is not equivariant.
  return extract_translation(multivectors).squeeze()
100%|██████████| 1000/1000 [06:27<00:00,  2.58it/s]


In [26]:
# from tqdm import tqdm
# from train import train, valid

# import torch
# import matplotlib.pyplot as plt

# # 初始化列表
# train_epoch_error = []
# val_epoch_error   = []
# test_epoch_error  = []

# train_check_idx = []          # 每轮 train loss 的横轴
# valid_check_idx = []          # 每 200 轮 val loss/test loss 的横轴

# epochs = 1000  # 假设训练 1000 轮
# interval = 200  # 每 200 轮评估一次 valid/test

# for epoch in tqdm(range(epochs), desc="Training"):
#     # —— 训练 —— 
#     tr_loss = train(model, device, train_loader, optimizer, criterion, scheduler=schedule)
#     train_epoch_error.append(torch.as_tensor(tr_loss))
#     train_check_idx.append(epoch + 1)

#     # —— 每 interval 个 epoch 执行一次验证和测试 —— 
#     if (epoch + 1) % interval == 0:
#         # —— 验证 —— 
#         val_loss = valid(model, device, valid_loader, criterion)
#         val_epoch_error.append(torch.as_tensor(val_loss))

#         # —— 测试 —— 
#         test_loss_list = []
#         model.eval()
#         with torch.no_grad():
#             for batch in test_loader:
#                 batch = batch.to(device)
#                 output = model(batch)
#                 label = batch.y
#                 loss = criterion(output, label)
#                 test_loss_list.append(loss.item())
#         avg_test_loss = sum(test_loss_list) / len(test_loss_list)
#         test_epoch_error.append(avg_test_loss)

#         # —— 记录当前 step —— 
#         valid_check_idx.append(epoch + 1)

#         # —— 保存当前模型（你也可以只保存最优的） —— 
#         torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'val_loss': val_loss,
#             'test_loss': avg_test_loss
#         }, f'checkpoint_epoch{epoch+1}.pt')


# # —— 绘图 —— 
# fig, ax = plt.subplots(figsize=(10, 8))

# # 转换为 numpy
# train_loss_np = torch.stack(train_epoch_error).detach().cpu().numpy()
# val_loss_np   = torch.stack(val_epoch_error).detach().cpu().numpy()
# test_loss_np  = torch.tensor(test_epoch_error).detach().cpu().numpy()

# # 画 train loss（每一轮都有）
# ax.plot(train_check_idx, train_loss_np, label='L_train', linewidth=1)

# # 画 valid loss（每 200 轮）
# ax.plot(valid_check_idx, val_loss_np, label='L_valid', linewidth=2, marker='o')

# # 画 test loss（每 200 轮）
# ax.plot(valid_check_idx, test_loss_np, label='L_test', linewidth=2, marker='x')

# ax.set_yscale('log')
# ax.set_title('Train / Valid / Test Loss Curve')
# ax.set_xlabel('Epochs')
# ax.set_ylabel('Loss')
# ax.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()



In [27]:
# from scipy.stats import spearmanr
# import numpy as np
# import torch

# per_point_diff_list = []
# disp_list = []
# mse_list = []
# all_preds = []
# all_labels = []

# for batch in valid_loader: 
#     batch = batch.to(device)
#     with torch.no_grad():
#         output = model(batch)
#         label  = batch.y

#         # ---- 反归一化 ----
#         pressure_normalizer = Normalizer_ts(method="ms",
#                                             params=output_range[0, :2].reshape(2, 1))
#         output_denorm = pressure_normalizer.denormalize(output.cpu()[:, 0])
#         label_denorm  = pressure_normalizer.denormalize(label .cpu()[:, 0])

#         # ---- 三个表 3 指标 ----
#         diff = output_denorm - label_denorm
#         disp = torch.sum(diff ** 2) / torch.sum(output_denorm ** 2)   # Approx. Disp.
#         mse  = torch.mean(diff ** 2)                                  # Test loss (MSE)

#         per_point_diff_list.append(diff.cpu().numpy())
#         disp_list.append(disp.item())
#         mse_list.append(mse.item())
#         all_preds.append(output_denorm.cpu().numpy())
#         all_labels.append(label_denorm.cpu().numpy())

# # ----------------- 汇总到整个验证集 -----------------
# preds   = np.concatenate(all_preds)
# labels  = np.concatenate(all_labels)
# diff_all = preds - labels

# # 原有 3 项
# per_point_diff_mean = diff_all.mean()      # Per-point Diff.
# disp_mean           = np.mean(disp_list)   # Approx. Disp.
# mse_mean            = np.mean(mse_list)    # Test loss (MSE)

# # # 新增 ρ（Spearman）-------------------------------
# # rho, _ = spearmanr(preds, labels)          # 若只看狭窄区，可先筛选再算

# # # 新增 Bias ------------------------------
# # bias = diff_all.mean()


# # ----------------- 打印 -----------------
# print(f"{'Model':<12}"
#       f"{'Test loss (Pa × 10⁻²) ↓':>20}"
#       f"{'Per-point Diff. ↓':>20}"
#       f"{'Approx. Disp. ↓':>18}"
#     #   f"{'ρ ↑':>8}"
#     #   f"{'Bias ↓':>12}"
#       )
# print('-' * 115)
# print(f"{'LaB-GATr':<12}"
#       f"{mse_mean * 100:>20.2f}"          # ×10⁻² 缩放
#       f"{per_point_diff_mean:>20.2E}"
#       f"{disp_mean:>18.2E}"
#     #   f"{rho:>8.3f}"
#     #   f"{bias:>12.3E}"
#       )


In [28]:
# import torch
# import pyvista as pv
# import matplotlib.pyplot as plt
# from os.path import join as osj
# from tqdm import tqdm


# def generate_output(
#     model,
#     device,
#     output_loader,
#     raw_mesh_path,
#     tag,
#     save_mesh_path,
#     mesh_name="mesh",
#     output_normalizer=None,
# ):
#     for _, batch in enumerate(tqdm(output_loader)):
#         batch = batch.to(device)
#         with torch.no_grad():
#             label = batch.y
#             output = model(batch)
#             label_denorm = output_normalizer.denormalize(label)  # shape is <N,1>
#             output_denorm = output_normalizer.denormalize(output)  # shape is <N,1>
#             idx = (batch.idx)[0]

#             mesh = pv.read(osj(raw_mesh_path, "aorta00{:d}.vtp".format(idx)))
#             mesh.point_data[tag + "_prediction"] = output_denorm.detach().cpu()
#             mesh.point_data[tag + "_label"] = label_denorm.detach().cpu()
#             mesh.point_data[tag + "_error"] = (
#                 mesh.point_data[tag + "_prediction"] - mesh.point_data[tag + "_label"]
#             )
#             mesh.save(osj(save_mesh_path, mesh_name + "_{:d}.vtp".format(idx)))
# generate_output(model_opt,'cpu',valid_loader,'aorta/raw','wss','results/meshes','mesh_wss_50',output_normalizer)